In [1]:
# Importing packages and loading data
import numpy as np
data = np.loadtxt("spam.data")
np.random.shuffle(data)

In [2]:
class DecisionTree():
    
    def __init__(self):
        self.tree = {}
        
    def learn(self,train_data,m_):
        
        self.tree['attribute'] = np.nan
        self.tree['threshold'] = np.nan
        
        records, no_of_attributes = train_data.shape
        no_of_attributes -= 1
        attributes_used = np.random.choice(int(no_of_attributes),size=m_)
        p = np.count_nonzero(train_data[:,-1])/records
        if p == 1:
            a = {}
            a['attribute'] = -1
            return a
        elif p == 0:
            a = {}
            a['attribute'] = -2
            return a
        min_entropy = 1
        spam = np.count_nonzero(train_data[:,-1])
        non_spam = records-spam
        for i in attributes_used:
            no_of_thresholds = 10
            m = np.min(train_data[:,i])
            M = np.max(train_data[:,i])
            thresholds = np.linspace(m+(M-m)/(no_of_thresholds+1),M,no_of_thresholds,endpoint=False)
            for j in range(no_of_thresholds):
                difference = train_data[:,i] - thresholds[j]
                Pass_spam = 0
                Pass_non_spam = 0
                for k in range(records):
                    if difference[k] > 0:
                        if train_data[k,-1] == 1:
                            Pass_spam += 1
                        else:
                            Pass_non_spam += 1
                Fail_spam = spam-Pass_spam
                Fail_non_spam = non_spam-Pass_non_spam
                Pass = Pass_spam+Pass_non_spam
                Fail = records-Pass
                
                if Pass_spam == 0 or Pass_non_spam == 0:
                    left_entropy = 0
                else:
                    left_entropy = -Pass_spam/Pass*np.log(Pass_spam/Pass)-Pass_non_spam/Pass*np.log(Pass_non_spam/Pass)
                
                if Fail_spam == 0 or Fail_non_spam == 0:
                    right_entropy = 0
                else:
                    right_entropy = -Fail_spam/Fail*np.log(Fail_spam/Fail)-Fail_non_spam/Fail*np.log(Fail_non_spam/Fail)
                               
                entropy = left_entropy*Pass/(Pass+Fail)+right_entropy*Fail/(Pass+Fail)
                if entropy <= min_entropy:
                    min_entropy = entropy
                    self.tree['attribute'] = i
                    self.tree['threshold'] = thresholds[j]
        
        Pass_indices = []
        Fail_indices = []
        for x in range(records):
            if train_data[x,self.tree['attribute']] - self.tree['threshold'] > 0:
                Pass_indices.append(x)
            else:
                Fail_indices.append(x)
        
        if len(Pass_indices) == 0 or len(Fail_indices) == 0:
            if p > 0.5:
                self.tree['attribute'] = -1
            else:
                self.tree['attribute'] = -2
            return self.tree
        
        left_subtree = DecisionTree()
        right_subtree = DecisionTree()
        self.tree['left_subtree'] = left_subtree.learn(train_data[Pass_indices,:],m_)
        self.tree['right_subtree'] = right_subtree.learn(train_data[Fail_indices,:],m_)
        return self.tree
    
    def classify(self, record, tree=None):
        if tree == None:
            tree = self.tree
        if tree['attribute'] == -1:
            return 1
        elif tree['attribute'] == -2:
            return 0
        else:
            if record[tree['attribute']] > tree['threshold']:
                return self.classify(record, tree['left_subtree'])
            else:
                return self.classify(record, tree['right_subtree'])
            

In [3]:
class RandomForest():
    
    def __init__(self):
        self.dec_tree = []
        self.records = []
        self.no_of_dec_trees = 100
        for i in range(self.no_of_dec_trees):
            self.dec_tree.append(DecisionTree())
        
    def learn(self, train_data):
        
        m = train_data.shape[1]//3
        for i in range(self.no_of_dec_trees):
            records_used = np.random.choice(train_data.shape[0], train_data.shape[0]//10)
            self.records.append(records_used)
            temp = train_data[records_used,:]
            self.dec_tree[i].learn(temp,m)
        self.OOBerror(train_data)
        
    def classify(self, record):
        preds = []
        for i in range(self.no_of_dec_trees):
            preds.append(self.dec_tree[i].classify(record))
        if sum(preds) > len(preds)//2:
            return 1
        else:
            return 0
        
    def OOBerror(self, train_data):
        correct_preds = 0
        for k in range(len(train_data)):
            preds = []
            for i in range(self.no_of_dec_trees):
                if k in self.records[i]:
                    continue
                preds.append(self.dec_tree[i].classify(train_data[k,:])-train_data[k,-1])
            if len(preds) > 0:
                if np.count_nonzero(preds)/len(preds) < 1/2:
                    correct_preds += 1
        print("Out of bag error =", 1-correct_preds/len(train_data))

In [4]:
%%time
# Performing Stratified sampling on the data
data = data[data[:,-1].argsort()]
spam = np.count_nonzero(data[:,-1])
non_spam_data = data[:data.shape[0]-spam,:]
spam_data = data[data.shape[0]-spam:,:]
non_spam_train = non_spam_data[:(non_spam_data.shape[0]*7)//10,:]
non_spam_test = non_spam_data[(non_spam_data.shape[0]*7)//10:,:]
spam_train = spam_data[:(spam_data.shape[0]*7)//10,:]
spam_test = spam_data[(spam_data.shape[0]*7)//10:,:]
train_data = np.concatenate((non_spam_train,spam_train),axis=0)
test_data = np.concatenate((non_spam_test,spam_test),axis=0)
np.random.shuffle(train_data)

# Learning and evaluating the model
a = RandomForest()
a.learn(train_data)
predictions = np.zeros(len(test_data))
for i in range(len(test_data)):
    predictions[i] = a.classify(test_data[i,:-1])
accuracy = 1-np.count_nonzero(predictions - test_data[:,-1])/len(test_data)
print("accuracy = ", accuracy)

Out of bag error = 0.07080745341614902
accuracy =  0.9355539464156408
CPU times: user 33.2 s, sys: 21.1 ms, total: 33.2 s
Wall time: 33.2 s


In [5]:
%%time
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(train_data[:,:-1],train_data[:,-1])
clf.score(test_data[:,:-1],test_data[:,-1])

CPU times: user 992 ms, sys: 700 ms, total: 1.69 s
Wall time: 801 ms


0.9478638667632151